In [0]:

%pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 MB 206.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 178.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 151.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 21.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from transformers import pipeline
import mlflow


/local_disk0/.ephemeral_nfs/envs/pythonEnv-e1fcb4b6-a796-4390-9c98-fbac2194d501/lib/python3.12/site-packages/torch/_vmap_internals.py:9: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  from torch.utils._pytree import _broadcast_to_and_flatten, tree_flatten, tree_unflatten


# Build a simple NLP task

In [0]:
# Load sentiment analysis pipeline (Mosaic AI / HuggingFace compatible)
classifier = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [0]:
# Sample product reviews
reviews = [
    "This product is amazing and works perfectly!",
    "Terrible quality, totally disappointed",
    "Good value for money",
    "Worst purchase I have ever made"
]

results = classifier(reviews)
results


[{'label': 'POSITIVE', 'score': 0.9998794794082642},
 {'label': 'NEGATIVE', 'score': 0.9997342228889465},
 {'label': 'POSITIVE', 'score': 0.9998501539230347},
 {'label': 'NEGATIVE', 'score': 0.9997796416282654}]

# Create AI-powered insights

In [0]:
sentiment_df = spark.createDataFrame(
    [(reviews[i], results[i]['label'], results[i]['score']) for i in range(len(reviews))],
    ["review_text", "sentiment", "confidence"]
)

sentiment_df.display()


review_text,sentiment,confidence
This product is amazing and works perfectly!,POSITIVE,0.9998794794082642
"Terrible quality, totally disappointed",NEGATIVE,0.9997342228889465
Good value for money,POSITIVE,0.9998501539230347
Worst purchase I have ever made,NEGATIVE,0.9997796416282654


# Mosaic AI Exploration (MLflow logging)

In [0]:
with mlflow.start_run(run_name="sentiment_analysis_model"):
    mlflow.log_param("model", "distilbert-base-uncased-finetuned-sst-2")
    mlflow.log_metric("avg_confidence", 
                      sum([r["score"] for r in results]) / len(results))
